In [1]:
# Import libraries:
import pandas as pd
import matplotlib.pyplot as plt
import dt_ids7_export_utils as bh_utils

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Check for missing time data:
df_ids7 = bh_utils.filter_NaT(df_ids7, True)
# Check for cancelled procedures:
df_ids7 = bh_utils.filter_cancelled(df_ids7, True)
# Check for non-human subjects (e.g. phantoms, animals, etc.):
df_ids7 = bh_utils.filter_phantom_etc(df_ids7, True)
# Check for invalid accession numbers:
df_ids7 = bh_utils.check_accession_format(df_ids7, True)
# Check whether an accession number is in the DoseTrack data:
df_ids7 = bh_utils.check_accession_ids7_vs_dt(df_ids7, df_dt, True)

Number of rows with NaT in the column "Bestilt dato og tidspunkt": 1
Number of cancelled procedures: 393
Number of non-human subjects: 7
Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2218
Number of accession numbers in IDS7 not in DoseTrack: 136


In [14]:
# Now we must go through the data and find all patients with different accession numbers at the dame time:
# We must then check whether any of these do not have corresponding dosetrack accession number.
# If so, we must overwrite the accession number with the one that allready exists in dosetrack.
# If there is multiple accession numbers at the same time that exists in dosetrack and one that does not, we must print the patient number, time and accession number
# to alert the user that data for this patient must be double checked. 

patient_list = df_ids7['Pasient'].unique()
for patient in patient_list:
    # Get the booking times for this patient in ascending order:
    booking_times = sorted(df_ids7[df_ids7['Pasient'] == patient]['Bestilt dato og tidspunkt'].unique())
    # Go thought the booking times for this patient:
    for time in booking_times:
        # Get the accession number for this patient at this booking time:
        acc_nr = df_ids7[(df_ids7['Pasient'] == patient) & (df_ids7['Bestilt dato og tidspunkt'] == time)]['Accession number'].unique()
        # Check if there is more than one accession number for this patient at this time:
        if len(acc_nr) > 1:
            


AttributeError: 'DatetimeArray' object has no attribute 'sort_values'

In [25]:
print(df_ids7.keys())


Index(['Prioritet- og lesemerkeikon', 'Lagt til i demonstrasjon-ikon',
       'Bestilt dato og tidspunkt', 'Pasient', 'Avbrutt',
       'Henvisningskategori (RIS)', 'Status', 'Kjønn', 'Gjeldende alder',
       'Kode', 'Beskrivelse', 'Rom/modalitet (RIS)', 'Henvisnings-ID',
       'Undersøkelses-ID', 'Beskrivelsens forfatter', 'Henvisning_i_dt'],
      dtype='object')


In [23]:
booking_times = sorted(df_ids7['Bestilt dato og tidspunkt'].unique())
print(booking_times)

[Timestamp('2023-01-01 11:32:00'), Timestamp('2023-01-02 07:00:00'), Timestamp('2023-01-02 08:00:00'), Timestamp('2023-01-02 08:30:00'), Timestamp('2023-01-02 08:45:00'), Timestamp('2023-01-02 10:15:00'), Timestamp('2023-01-02 10:45:00'), Timestamp('2023-01-02 12:30:00'), Timestamp('2023-01-02 15:00:00'), Timestamp('2023-01-02 17:38:00'), Timestamp('2023-01-03 07:00:00'), Timestamp('2023-01-03 09:45:00'), Timestamp('2023-01-03 10:45:00'), Timestamp('2023-01-03 11:19:00'), Timestamp('2023-01-03 12:30:00'), Timestamp('2023-01-03 13:30:00'), Timestamp('2023-01-03 15:00:00'), Timestamp('2023-01-03 17:36:00'), Timestamp('2023-01-04 07:00:00'), Timestamp('2023-01-04 08:00:00'), Timestamp('2023-01-04 08:45:00'), Timestamp('2023-01-04 12:00:00'), Timestamp('2023-01-04 13:15:00'), Timestamp('2023-01-04 15:00:00'), Timestamp('2023-01-04 16:02:00'), Timestamp('2023-01-04 18:19:00'), Timestamp('2023-01-05 07:00:00'), Timestamp('2023-01-05 11:15:00'), Timestamp('2023-01-05 14:07:00'), Timestamp('20

In [12]:
# Check if there are any rows with false values in the 'Henvisning_i_dt' column:
if df_ids7['Henvisning_i_dt'].any() == False:
    print('All accession numbers are in DoseTrack.')



We need to create logic for merging lines in the ids7 data:

During a booking the patient gets an accession number:
As the procedure moves forward more codes can be added to the same accession number. These codes should be merged into one line.

Check if the accession number exists in Dosetrack. If it does, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does not have a corresponding entry in dosetrack. If so they might be in the same procedure.

Alternatively, if there are an accession number that is not included in doseTrack, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does have a corresponding entry in dosetrack. If so they might be in the same procedure.

If neither of the procedures exists in dosetrack they might or might not be the same procedure.

Print a list of all the changes performed on the data.

In [ ]:
# Check for patients with multiple bookings on the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_time_with_different_accession(df_ids7)

In [6]:
# Check for patients with multiple bookings on the same day but not the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_day_with_different_accession(df_ids7)